# qiime2 analysis

## 1.qc

In [1]:
!qiime metadata tabulate \
  --m-input-file sample-metadata1.tsv \
  --o-visualization qiime_visualization/metadata-summ-1.qzv

Saved Visualization to: qiime_visualization/metadata-summ-1.qzv


In [4]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --input-path data_to_import \
  --output-path qiime_result/qc/demultiplexed-sequences.qza

Imported data_to_import as CasavaOneEightSingleLanePerSampleDirFmt to qiime_result/qc/demultiplexed-sequences.qza


In [5]:
!qiime demux summarize \
  --i-data qiime_result/qc/demultiplexed-sequences.qza \
  --o-visualization qiime_visualization/demultiplexed-sequences-summ.qzv

Saved Visualization to: qiime_visualization/demultiplexed-sequences-summ.qzv


## 2.denoise-paired

In [6]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs qiime_result/qc/demultiplexed-sequences.qza \
  --p-trim-left-f 0 \
  --p-trunc-len-f 220 \
  --p-trim-left-r 0 \
  --p-trunc-len-r 220 \
  --p-n-threads 3 \
  --o-representative-sequences qiime_result/qc/asv-sequences-0.qza \
  --o-table qiime_result/qc/feature-table-0.qza \
  --o-denoising-stats qiime_result/qc/dada2-stats.qza

Saved FeatureTable[Frequency] to: qiime_result/qc/feature-table-0.qza
Saved FeatureData[Sequence] to: qiime_result/qc/asv-sequences-0.qza
Saved SampleData[DADA2Stats] to: qiime_result/qc/dada2-stats.qza


In [7]:
!qiime metadata tabulate \
  --m-input-file qiime_result/qc/dada2-stats.qza \
  --o-visualization qiime_visualization/dada2-stats-summ.qzv

Saved Visualization to: qiime_visualization/dada2-stats-summ.qzv


## 3.filtered and classification

In [8]:
!qiime feature-table summarize \
  --i-table qiime_result/qc/feature-table-0.qza \
  --m-sample-metadata-file sample-metadata1.tsv \
  --o-visualization qiime_visualization/feature-table-0-summ.qzv
!qiime feature-table tabulate-seqs \
  --i-data qiime_result/qc/asv-sequences-0.qza \
  --o-visualization qiime_visualization/asv-sequences-0-summ.qzv

Saved Visualization to: qiime_visualization/feature-table-0-summ.qzv
Saved Visualization to: qiime_visualization/asv-sequences-0-summ.qzv


In [9]:
!qiime feature-classifier classify-sklearn \
  --i-classifier qiime_result/gg-13-8-99-nb-classifier.qza \
  --i-reads qiime_result/qc/asv-sequences-0.qza \
  --o-classification qiime_result/taxon/taxonomy.qza

Saved FeatureData[Taxonomy] to: qiime_result/taxon/taxonomy.qza


In [10]:
!qiime metadata tabulate \
  --m-input-file qiime_result/taxon/taxonomy.qza \
  --o-visualization qiime_visualization/taxonomy.qzv

Saved Visualization to: qiime_visualization/taxonomy.qzv


In [11]:
!qiime taxa filter-table \
  --i-table qiime_result/qc/feature-table-0.qza \
  --i-taxonomy qiime_result/taxon/taxonomy.qza \
  --p-mode contains \
  --p-include p__ \
  --p-exclude 'p__;,Chloroplast,Mitochondria' \
  --o-filtered-table qiime_result/filtered/filtered-table.qza

Saved FeatureTable[Frequency] to: qiime_result/filtered/filtered-table.qza


In [12]:
!qiime feature-table filter-seqs \
  --i-data qiime_result/qc/asv-sequences-0.qza \
  --i-table qiime_result/filtered/filtered-table.qza \
  --o-filtered-data qiime_result/filtered/filtered-sequences.qza

Saved FeatureData[Sequence] to: qiime_result/filtered/filtered-sequences.qza


In [13]:
!qiime taxa barplot \
  --i-table qiime_result/filtered/filtered-table.qza \
  --i-taxonomy qiime_result/taxon/taxonomy.qza \
  --m-metadata-file sample-metadata1.tsv \
  --o-visualization qiime_visualization/taxa-bar-plots-1.qzv

Saved Visualization to: qiime_visualization/taxa-bar-plots-1.qzv


In [14]:
!qiime tools export \
  --input-path qiime_result/filtered/filtered-table.qza \
  --output-path qiime_result/filtered/filtered-table
!qiime tools export \
  --input-path qiime_result/filtered/filtered-sequences.qza \
  --output-path qiime_result/filtered/filtered-sequences
!biom convert -i qiime_result/filtered/filtered-table/feature-table.biom \
    -o qiime_result/filtered/filtered-table/feature-table.txt \
    --to-tsv

Exported qiime_result/filtered/filtered-table.qza as BIOMV210DirFmt to directory qiime_result/filtered/filtered-table
Exported qiime_result/filtered/filtered-sequences.qza as DNASequencesDirectoryFormat to directory qiime_result/filtered/filtered-sequences


In [15]:
import pandas as pd
data=pd.read_csv('qiime_result/filtered/filtered-table/feature-table.txt',sep='\t',skiprows=1,index_col=0)
data.index.name='sequence'
data.to_csv('qiime_result/filtered/filtered-table/feature-table.tsv',sep='\t')

In [16]:
!qiime feature-classifier classify-sklearn \
  --i-reads qiime_result/filtered/filtered-sequences.qza \
  --i-classifier classify/bespoke.qza \
  --o-classification qiime_result/taxon/bespoke_taxonomy.qza

!qiime metadata tabulate \
  --m-input-file qiime_result/taxon/bespoke_taxonomy.qza \
  --o-visualization qiime_visualization/bespoke_taxonomy.qzv

!qiime taxa barplot \
  --i-table qiime_result/filtered/filtered-table.qza \
  --i-taxonomy qiime_result/taxon/bespoke_taxonomy.qza \
  --m-metadata-file sample-metadata1.tsv \
  --o-visualization qiime_visualization/bespoke_taxa_barplot.qzv

Saved FeatureData[Taxonomy] to: qiime_result/taxon/bespoke_taxonomy.qza
Saved Visualization to: qiime_visualization/bespoke_taxonomy.qzv
Saved Visualization to: qiime_visualization/bespoke_taxa_barplot.qzv


## 4. Pathway and function prediction

In [17]:
!qiime vsearch cluster-features-closed-reference \
  --i-table qiime_result/filtered/filtered-table.qza \
  --i-sequences qiime_result/filtered/filtered-sequences.qza \
  --i-reference-sequences qiime_result/otu/85_otus.qza \
  --p-perc-identity 0.85 \
  --o-clustered-table qiime_result/otu/table-cr-85.qza \
  --o-clustered-sequences qiime_result/otu/rep-seqs-cr-85.qza \
  --o-unmatched-sequences qiime_result/otu/unmatched-cr-85.qza

Saved FeatureTable[Frequency] to: qiime_result/otu/table-cr-85.qza
Saved FeatureData[Sequence] to: qiime_result/otu/rep-seqs-cr-85.qza
Saved FeatureData[Sequence] to: qiime_result/otu/unmatched-cr-85.qza


In [18]:
!qiime tools export \
  --input-path qiime_result/otu/table-cr-85.qza \
  --output-path qiime_result/otu/table-cr-85
!qiime tools export \
  --input-path qiime_result/otu/rep-seqs-cr-85.qza \
  --output-path qiime_result/otu/rep-seqs-cr-85
!biom convert -i qiime_result/otu/table-cr-85/feature-table.biom \
    -o qiime_result/otu/table-cr-85/feature-table.txt \
    --to-tsv

Exported qiime_result/otu/table-cr-85.qza as BIOMV210DirFmt to directory qiime_result/otu/table-cr-85
Exported qiime_result/otu/rep-seqs-cr-85.qza as DNASequencesDirectoryFormat to directory qiime_result/otu/rep-seqs-cr-85


In [19]:
import pandas as pd
data=pd.read_csv('qiime_result/otu/table-cr-85/feature-table.txt',sep='\t',skiprows=1,index_col=0)
data.index.name='sequence'
data.to_csv('qiime_result/otu/table-cr-85/feature-table.tsv',sep='\t')

In [20]:
!MicFunPred_run_pipeline.py -i qiime_result/otu/table-cr-85/feature-table.tsv \
  -r qiime_result/otu/rep-seqs-cr-85/dna-sequences.fasta \
  -o qiime_result/otu/micfunpred_cr_85_contrib \
  --verbose \
  --contrib

Running BLAST with 98.3 cut-off.
Predicting 16S rRNA copy numbers
Predicting KO copy numbers
Predicting KO metagenome
Running MinPath (KO)
Anotating predicted metagenome
/Users/fernandozeng/miniforge3/envs/qiime2-amplicon-2023.9/lib/python3.8/site-packages/micfunpreDefinitions/micfunpreDefinitions.py:391: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_ko = df_ko.groupby('C').sum()
/Users/fernandozeng/miniforge3/envs/qiime2-amplicon-2023.9/lib/python3.8/site-packages/micfunpreDefinitions/micfunpreDefinitions.py:157: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  return metagenomeDf.groupby(group).sum()
/Users